
# Hoofdstuk 15 - Databases

In de slides behorende bij hoofdstuk 15 wordt uitgebreid ingegaan op SQL en op database normalisatie. Als het goed is weten jullie dat allemaal nog van het vak Computer Forensics & Gegevensbeheer wat vorig jaar gegeven is.

Helaas staan in de slides geen Python voorbeelden hoe je SQL queries vanuit je eigen script kunt aanroepen.

# Gebruik slides

Als je deze slides thuis gebruik je deze knop op Blackboard: [![Binder](badge_logo.png)](https://mybinder.org/v2/gh/richardbrinkman/pythonlearn15/master?filepath=slides-addons.ipynb)

* met de spatiebalk ga je naar de volgende slide
* met shift+spatiebalk ga je naar de vorige slide
* code blokken kun je zelf aanpassen
* code blok kun je uitvoeren met Shift-Enter

## Initialisatie

Voor iedere database is er wel een Python module te vinden om hier mee te connecten. Deze zullen allemaal heel erg op elkaar lijken. In deze notebook zullen we gebruik maken van de bij jullie reeds bekende SQlite.

Allereerst moeten we de sqlite3 module importeren in ons script. Er is verder geen noodzaak om hiervoor iets te moeten installeren aangezien elke Python interpreter reeds wordt meegeleverd met deze module.

In [ ]:
import sqlite3

Vervolgens moet een database bestand worden geopend. Vergelijk dit met het openen van een tekstbestand met de `open()` functie. We gebruiken hiervoor de `sqlite3.connect()` functie voor.

In [ ]:
connection = sqlite3.connect('example.db')

In [ ]:
type(connection)

## Bevragen van een database connectie

In [ ]:
help(sqlite3.Connection.execute)

Veruit de meeste database modules gebruiken een `cursor` object.

In [ ]:
cursor = connection.cursor()

In [ ]:
type(cursor)

In [ ]:
help(sqlite3.Cursor.execute)

In [ ]:
cursor.execute('SELECT * FROM Users')

Je verwacht op dit punt waarschijnlijk de inhoud van de tabel te zien, zoals je bij de sqlite command line tool gewend was.

De SQL query is weliswaar uitgevoerd, maar de uitkomst moet nog wel worden uitgelezen in Python, bijvoorbeeld door een `for` lusje.

In [ ]:
for row in cursor:
    print(row)

Zoals je ziet gedraagt een `cursor` object zich (binnen een `for` lus) als een lijst van `tuple`'s.

Maar een `cursor` object kan veel meer:

In [ ]:
help(sqlite3.Cursor)

In plaats van de `for` lus kan hetzelfde effect verkregen worden d.m.v. de `fetchall()` functie.

In [ ]:
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

Je kunt ook één voor één de rows opvragen:

In [ ]:
cursor.execute('SELECT * FROM Users')
cursor.fetchone()

In [ ]:
cursor.fetchone()

Het restant wat dan overblijft is dan:

In [ ]:
cursor.fetchall()

De elementen van een cursor zijn dus maar eenmalig op te vragen. Heb je ze dus vaker nodig in je Python script dan zul je de lijst in een variabele moeten opslaan.

Als een cursor alle elementen heeft ge`fetch`ed, dan blijft er een lege lijst over:

In [ ]:
cursor.fetchall()

In [ ]:
print(cursor.fetchone())

# `fetchone()` / `for` versus `fetchall()`

Bij zeer grote datasets is het verstandiger om `fetchone()` of een `for` lus te gebruiken aangezien bij `fetchall()` alles in één keer wordt gedownload en dus in het geheugen van de client moet passen.

## INSERT queries

Een enkele regel toevoegen aan een tabel gaat als volgt:

In [ ]:
cursor.execute('INSERT INTO Users(name, email) VALUES(?, ?)', 
               ('Richard Brinkman', 'r.brinkman@saxion.nl'))

Dat kunnen we checken met:

In [ ]:
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

### SQL injection attack

Zoals je ziet gebruiken we placeholders i.p.v. direct een SQL query als

```sql
INSERT INTO Users(name, email) 
VALUES ('Richard Brinkman', 
        'r.brinkman@saxion.nl')
```

Als de data die je wilt inserten namelijk afkomstig is van een onbetrouwbare bron (bijvoorbeeld hacker die een formuliertje invoert op een website), wil je deze niet direct in de query zetten.

Beschouw deze gevaarlijke code:

In [ ]:
name = input('Wat is uw naam: ')
sql = "INSERT INTO Users(name) VALUES ('" + name + "');"
cursor.executescript(sql)

Even checken met:

In [ ]:
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

Zolang je "normale" data opgeeft gaat het weliswaar goed, maar wat als de gebruiker iets intypt als:

```sql
Brinkman'); DELETE FROM Users WHERE name = 'Ted' AND ('' == '
```

Je zou dan 2 queries uitvoeren namelijk:
```sql
INSERT INTO Users(name) 
            VALUES ('Brinkman');
DELETE FROM Users WHERE name = 'Ted' AND 
                        ('' == '');
```

In [ ]:
name = input('Wat is uw naam: ')
sql = "INSERT INTO Users(name) VALUES ('" + name + "');"
cursor.executescript(sql)
print(sql)

Even checken wat er is gebeurd:

In [ ]:
# Ted is verdwenen !!!
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

Gebruik daarom *altijd* placeholders als een deel van je query komt van een variabele.

De correcte manier is:

In [ ]:
name = input('Wat is uw naam')
cursor.execute('INSERT INTO Users(name) VALUES (?)', (name,))

In [ ]:
cursor.execute("SELECT * FROM Users")
cursor.fetchall()

## Named placeholders
Naast de `?` placeholders kun je ook named placeholders gebruiken:

In [ ]:
data = {"name": "Jan", "email": "j.klaassen@gmail.com"}
cursor.execute("INSERT INTO Users(name, email) VALUES(:name, :email)", data)

## Meerdere regels tegelijk inserten

Stel we hebben al een lijst van gegevens. Deze kunnen we met één enkele SQL query invoeren:

In [ ]:
personen = [
    ('Brinkman', 'r.brinkman@saxion.nl'),
    ('van Tuinen', 'm.vantuinen@saxion.nl')
]
cursor.executemany('INSERT INTO Users(name, email) VALUES (?, ?)', 
                   personen)

Even checken:

In [ ]:
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

## Meerdere cursors

Soms zal het noodzakelijk zijn om meerdere queries parallel naast elkaar te draaien. Je kunt hiervoor meerdere cursors gebruiken

In [ ]:
# convert all email addresses to lower case
cursor1 = connection.cursor()
cursor2 = connection.cursor()
for email, in cursor1.execute('''SELECT email FROM Users 
                                 WHERE email IS NOT NULL'''):
    cursor2.execute('''UPDATE Users SET email = ? 
                       WHERE email = ?''', 
                    (email.lower(), email))

## Commit / Rollback

Alles wat je met een `connectie` object doet wordt niet direct weggeschreven naar schijf. Het bestand `example.db` is dus nog altijd niet veranderd; dat gebeurt pas na het volgende commando:

In [ ]:
connection.commit()

Als je alle queries in de transactie ongedaan maken kun je ook altijd

```python
connection.rollback()
```

draaien.

# Recap

* open eenmalig een database connectie met `connection = sqlite3.connect('bestandsnaam.db')`
* maak een cursor object aan met `cursor = connection.cursor()`
* gebruik één van de `execute...()` functies: `cursor.execute('SELECT ...')`

* gebruik één van de `fetch...()` functies om de data te downloaden: `cursor.fetchAll()` of gebruik de cursor in een `for row in cursor:` loopje
* `connection.commit()` om je data naar schijf weg te schrijven

* lees de slides van het boek door als herhaling van het vak Computer Forensics & Gegevensbeheer.